In [1]:
import tensorflow as tf
import math

In [2]:
MAX_SEQUENCE_LENGTH = 1000;

In [3]:
def sequence_lengths(sequences):
    lengths = tf.reduce_sum(tf.sign(sequences), axis = 1);
    return lengths;

def build_model(embedding_size = 100, lstm_size = 300, vocabulary_size = 100000):
    x1 = tf.placeholder(tf.int32, shape = [None, MAX_SEQUENCE_LENGTH]);
    x2 = tf.placeholder(tf.int32, shape = [None, MAX_SEQUENCE_LENGTH]);
    
    embeddings = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                      stddev=1.0 / math.sqrt(embedding_size)));
    embedding_layer_1 = tf.nn.embedding_lookup(embeddings, x1);
    embedding_layer_2 = tf.nn.embedding_lookup(embeddings, x2);

    lstm_cell_1 = tf.contrib.rnn.LSTMCell(lstm_size);
    lstm_cell_2 = tf.contrib.rnn.LSTMCell(lstm_size);

    with tf.variable_scope("lstm_1"):
        _, (lstm_1_c_state, lstm_1_m_state) = tf.nn.dynamic_rnn(cell=lstm_cell_1,
                                     dtype = tf.float32,
                                     sequence_length = sequence_lengths(x1),
                                     inputs = embedding_layer_1,
                                     time_major = False);
    
    with tf.variable_scope("lstm_2"):
        _, (lstm_2_c_state, lstm_2_m_state) = tf.nn.dynamic_rnn(cell = lstm_cell_2,
                                     dtype = tf.float32,
                                     sequence_length = sequence_lengths(x2),
                                     inputs = embedding_layer_2,
                                     time_major = False);
    
    lstm_outputs = tf.concat([lstm_1_m_state, lstm_2_m_state], axis = 1);
    W = tf.Variable(tf.truncated_normal([2 * lstm_size, 2], stddev = 1.0/ math.sqrt(lstm_size)));
    b = tf.Variable(tf.truncated_normal([2], stddev = 1.0 / math.sqrt(2)));
    softmax_inputs = tf.matmul(lstm_outputs, W) + b;
    softmax_outputs = tf.nn.softmax(softmax_inputs);
    return x1, x2, softmax_outputs

In [4]:
x1, x2, softmax_outputs = build_model();
y_ = tf.placeholder(tf.float32, shape = (2,None));

In [5]:
def create_loss(outputs, y_):
    log_loss = tf.reduce_mean(tf.reduce_sum(tf.log(outputs) * y_, axis = 1));
    return log_loss;

In [6]:
log_loss = create_loss(softmax_outputs, y_)
print(log_loss.get_shape())

()


In [7]:
def create_optimizer(loss):
    optimizer = tf.train.AdamOptimizer()
    minimize = optimizer.minimize(loss)
    return minimize

In [9]:
minimize = create_optimizer(log_loss)

In [ ]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

In [ ]:
batch_size = 100
epochs = 2
num_batches = len(sequences) / batch_size
for e in range(epochs):
    for i in range(num_batches):
        train_x1 = sequences[0][i * batch_size : (i + 1) * batch_size]
        train_x2 = sequences[1][i * batch_size : (i + 1) * batch_size]
        target = similarities[i * batch_size : (i + 1) * batch_size]
        sess.run(minimize, {x1: train_x1, x2: train_x2, y_: target})
    train_loss = sess.run(loss, {x1: sequences[0], x2: sequences[1], y_: similarities})
    test_loss = sess.run(loss, {x1 : test_sequences[0], x2 : test_sequences[1], y_ : test_similarities})
    print(print('Epoch {:2d}, train error {:3.1f}, test error {:3.1f}%'.format(e, train_loss, test_loss)))
sess.close()